In [1]:
import urllib.request, json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

In [2]:
version = 'preprocessed_1000sample'
df_posts = pd.read_csv(f'data/final_proj_data_{version}.csv')

In [5]:
def getAllType(username, contentType):
    n_posts = 500
    with urllib.request.urlopen(f"https://api.pushshift.io/reddit/search/{contentType}/?author={username}&sort=asc&size={n_posts}") as url:
        data = json.loads(url.read().decode())
        data = data['data']
    df_content = pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns')
    if len(df_content) == 0:
        return df_content
    created_utc_last = df_content.tail(1)['created_utc'].copy().reset_index()
    created_utc_last = created_utc_last['created_utc'][0]
    while len(data) > 0:
        with urllib.request.urlopen(f"https://api.pushshift.io/reddit/search/{contentType}/?author={username}&sort=asc&size={n_posts}&after={created_utc_last}") as url:
            data = json.loads(url.read().decode())
            data = data['data']
        df_content = df_content.append(pd.DataFrame.from_dict(pd.json_normalize(data), orient='columns'))
        created_utc_last = df_content.tail(1)['created_utc'].copy().reset_index()
        created_utc_last = created_utc_last['created_utc'][0]
    return df_content

In [14]:
# Build subreddit mappings
sub_mappings = {}
for username in set(df_posts['author']):
    df_comment = getAllType(username, 'comment').reset_index()
    df_submission = getAllType(username, 'submission').reset_index()
    df_comment_set = set(df_comment['subreddit']) if len(df_comment) > 0 else {}
    df_submission_set = set(df_submission['subreddit']) if len(df_submission) > 0 else {}
    sub_mappings[username] = {
        'comment': df_comment_set,
        'submission': df_submission_set
    }
#     print('got subreddits for', username)

with open(f'data/authorsubs_{version}.json', 'w') as fp:
    json.dump(sub_mappings, fp)

KeyboardInterrupt: 

In [13]:
print(set(df_submission['subreddit']))

{'Advice', 'teenagers', 'NSFW411'}
